---
# Projeto 2 - Ciência dos Dados

Nome: Daniel Pucciariello

Nome: Stephanie Liu

A proposta desse trabalho é avaliar se um tweet sobre o tema "Pastel" foi feito por qual tipo de usuário: um usuário que gosta de Pastel e teve uma boa experiência decorrente do fato dele ter comido o alimento ou somente demonstra gostar desse (classiicação 2), um usuário que desgoste de pastel ou teve uma má experiência ao se alimentar dele (classificação 1) ou um usuário cujo emprego da palavra pastel tenha sido utilizada para expressar um sentimento que não se relacione com o alimento de forma direta (irrelevante foi classificado como 0).

---

___
# Classificador automático de sentimento
## Preparando o ambiente no jupyter:

In [ ]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji
!pip install preprocessor
!pip install TextBlob
!pip install preprocessing
!pip install preprocessing.text
!pip install keyword_tokenize
!pip install remove_unbound_punct
!pip install remove_urls


In [ ]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import emoji
import preprocessor 
from textblob import TextBlob
import string
import preprocessing
# import ntlk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
analyser = SentimentIntensityAnalyzer()
# from preprocessing.text import keyword_tokenize, remove_unbound_punct, remove_urls

In [ ]:
print(sentiment_analyzer_scores('😍'))


In [ ]:
# import preprocessing.text as ptext
# from preprocessing.text import keyword_tokenize, remove_unbound_punct, remove_urls

# text_string = "important string at: http://example.com"

# clean_string = ptext.preprocess_text(text_string, [
#     remove_urls,
#     remove_unbound_punct,
#     keyword_tokenize
# ])

# import re
# from nltk.tokenize import word_tokenize
# from string import punctuation 
# from nltk.corpus import stopwords 

# class PreProcessTweets:
#     def __init__(self):
#         self._stopwords = set(stopwords.words('portuguese') + list(punctuation) + ['AT_USER','URL'])
        
#     def processTweets(self, list_of_tweets):
#         processedTweets=[]
#         for tweet in list_of_tweets:
#             processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
#         return processedTweets
    
#     def _processTweet(self, tweet):
#         tweet = tweet.lower() # convert text to lower-case
#         tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
#         tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
#         tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
#         tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
#         return [word for word in tweet if word not in self._stopwords]

In [ ]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed=text_subbed.replace("\n", " ")
    text_subbed=text_subbed.replace("…", " ")
    text_subbed=text_subbed.replace("@", " ")
    text_subbed=text_subbed.replace("rt", "")
    text_subbed=text_subbed.replace("https", "")
    
    return text_subbed
#tirar coisas com https

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

#Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

---

In [ ]:
data_treinamento = pd.read_excel("Excel_pastel.xlsx",sheet_name="Treinamento")

In [ ]:
gostam_raw = " ".join(data_treinamento[data_treinamento.Classe==2].Treinamento)
ngostam_raw = " ".join(data_treinamento[data_treinamento.Classe==1].Treinamento)
irrelevantes_raw = " ".join(data_treinamento[data_treinamento.Classe==0].Treinamento)

In [ ]:
irrelevantes = cleanup(irrelevantes_raw)
#irrelevantes

In [ ]:
tabela_relativa_irrelevantes = pd.Series(irrelevantes.lower().split()).value_counts(True)
tabela_absoluta_irrelevantes = pd.Series(irrelevantes.lower().split()).value_counts(False)


In [ ]:
tabela_relativa_gostam = pd.Series(gostam_raw.lower().split()).value_counts(True)
tabela_absoluta_gostam = pd.Series(gostam_raw.lower().split()).value_counts(False)

In [ ]:
tabela_absoluta_ngostam = pd.Series(ngostam_raw.lower().split()).value_counts(False)

In [ ]:
tudo=irrelevantes_raw + gostam_raw +ngostam_raw
tudo=cleanup(tudo)
tudo_absoluto=pd.Series(tudo.lower().split()).value_counts(False)

In [ ]:
data_teste = pd.read_excel("Excel_pastel.xlsx",sheet_name="Teste")

In [ ]:
tudo_raw = tudo.split()
tudo_sem_repeticao = set(tudo_raw)

#tudo_sem_repeticao

In [ ]:
#data_teste_idx= data_teste.set_index("Teste")
# vamos ter a nossa string relevante e string de não relevante
# vamos somar os dois = total
# vamos dar o split = lista total
# vamos jogaar a lista total no set --> conjunto de palavras unicas
# ao temos o len do set  -- numero das palavras unicas

# ao inves da freq relativa 

# gosta=((tabela_absoluta_gostam[i.split()]+1)/(count_palavras_gostam+d)).prod()
# ngosta=((tabela_absoluta_ngostam_irrelevantes[i.split()]+1)/(count_palavras_gostam+d)).prod()
lista=[]
valor_a = []
# -------- Variaveis que gostam 
gostam_limpo = cleanup(gostam_raw)
palavras_gostam = gostam_limpo.split()

# ------- Variaveis que não gostam
ngostam_limpo = cleanup(ngostam_raw)
palavras_ngostam = ngostam_limpo.split()

# ------- Variaveis Irrelevantes
irrelevantes_limpo = cleanup(irrelevantes_raw)
palavras_irrelevantes = irrelevantes_limpo.split()


# CONTANDO AS VARIAVEIS 
count_palavras_gostam = len(palavras_gostam)
count_palavras_ngostam = len(palavras_ngostam)
count_palavras_irrelevantes = len(palavras_irrelevantes)

d = len(tudo_sem_repeticao)

#fazer if pra verificar se a palavra eh inedita ou nao; se for inedita, vai ser igual ao termo abaixo

# n_existe = 1/(count_palavras_gostam+d)

#se der underflow, tem que fazer log 
for i in data_teste.Teste:
    gosta = 1
    ngosta = 1
    irrelevante =1
    for x in i.split():
        prob_g = 0
        prob_n = 0
        if x not in tabela_absoluta_gostam:
            prob_g= 1/(count_palavras_gostam+d)
            
        elif x in tabela_absoluta_gostam:
            prob_g=((tabela_absoluta_gostam[x]+1)/(count_palavras_gostam+d))
            
        if x not in tabela_absoluta_irrelevantes:
            prob_i= 1/(count_palavras_irrelevantes+d)
            
        elif x in tabela_absoluta_irrelevantes:
            prob_i=((tabela_absoluta_irrelevantes[x]+1)/(count_palavras_irrelevantes+d))
        
        if x not in tabela_absoluta_ngostam:
            prob_n= 1/(count_palavras_ngostam+d)
            
        elif x in tabela_absoluta_ngostam:
            prob_n=((tabela_absoluta_ngostam[x]+1)/(count_palavras_ngostam+d))
        
        
        gosta*=prob_g
        ngosta*=prob_n
        irrelevante*=prob_i
    if gosta>ngosta and gosta >irrelevante:
        valor=2
    elif gosta<ngosta and ngosta>irrelevante:
        valor=1
    else:
        valor=0
    lista.append(valor)
#print(valor_a)
    #a = tabela_absoluta_gostam[i.split()]+1

data_teste.Classe = lista

data_teste.to_excel("Classificador_3_classificações.xlsx")


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
errou1=len(data_teste.loc[(data_teste.Classe==2)&(data_teste.Modelo<=1)])
errou2=len(data_teste.loc[(data_teste.Classe==1)&(data_teste.Modelo==0)])
errou3=len(data_teste.loc[(data_teste.Classe==0)&(data_teste.Modelo>=1)])
errou4=len(data_teste.loc[(data_teste.Classe==1)&(data_teste.Modelo==2)])

total_errou = errou1 + errou2 + errou3 + errou4
acuracia=1-(total_errou/(len(data_teste)))

acuracia

___
### Concluindo

Realizamos a classificação de 499 tweets de acordo com 3 requesitos, de modo a criar uma base de treinamento. A partir disso, foi calculada a probabilidade de cada tweet ser de uma categoria ou de outra. Tal cálculo foi realizado se utilizando de uma técnica conhecida como laplace smoothing aliada ao conceito de classificador Naive Bayes e dos dados anteriormente obtidos pela classificação manualmente feita.
No dia em que puxamos os tweets para serem classifcados, fomos infelizes com o fato de um assunto não relacionado diretamente com o nosso tema ter sido razoavelmente frequente no twitter. Tal assunto foi o lançamento do iPhone 11 na cor verde pastel. De qualquer maneira, a acurácia da classificação foi maior do que 70%.

Além disso, não podemos usar o próprio classificador para gerar mais amostras de treinamento, pois não há a certeza que tal resultado esteja correto, temos apenas a probabilidade de estar correto. Deste modo, supondo que fosse utilizado o próprio classificador com a finalidade de gerar mais amostrar de treinamento, se estaria supondo que o mesmo estivesse correto, de maneira que iria se criar um ciclo de desajuste e "descalibramento" da inteligência artificial.

**Outras aplicações para o classificar do tipo Naive Bayes:**
O classificador do tipo Naive Bayes assume que os elementos probabilisticos que irão compor um evento cuja classificação irá ser realizada são independentes entre si. Na prática, isso significa que o cálculo da probabilidade final de acontecimento de determinado evento pode ser simplificada pela multiplicação da probabilidade de cada um dos elementos que o compõe. Aplicações plausíveis e utilizadas frequemente são os classificadores de SPAM de emails e classificadores de assunto em um conjuntos de notícias, por exemplo.